<a href="https://colab.research.google.com/github/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decoding_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, style="whitegrid", palette="pastel", color_codes=True)

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_predict, LeaveOneOut
import os
import csv

In [2]:
##Download data file, set name

!wget https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
!mv 136_Data_Final.csv\?raw\=true 136_Data_Final.csv

--2019-04-12 21:40:06--  https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-12 21:40:11--  https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-12 21:40:11--  https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv
Resolving raw.githubus

In [3]:
#Import data to dataframe
##Implement Random forest stuff here...
df = pd.read_csv('136_Data_Final.csv')
df.set_index('Subject')
df.head()

,Unnamed: 0,Subject,Block,Trial,Time,O2,O1,OZ,PZ,P4,...,FC3,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value
0,0,136,1,1,-200,-2.6587,-5.8283,-3.6200,-8.4912,-6.0398,...,-4.2465,-6.1961,-0.7449,-4.0477,-6.5453,-252.5419,70.5294,13.6076,-66.9374,1.0
1,1,136,1,1,-196,-3.3376,-5.6611,-3.5900,-8.8003,-6.9963,...,-3.2880,-5.5053,-0.7434,-3.1528,-4.8430,-218.5595,51.3878,25.2795,-68.4394,1.0
2,2,136,1,1,-192,-3.4338,-4.6322,-2.9857,-8.3836,-7.6261,...,-2.0604,-4.9363,-0.8046,-2.6793,-2.9363,-196.0244,68.5147,18.5178,-64.9081,1.0
3,3,136,1,1,-188,-3.6411,-3.6174,-2.5697,-7.9997,-8.4763,...,-1.1900,-5.0391,-0.8101,-2.7765,-1.7893,-191.9580,51.4932,22.3986,-66.2655,1.0
4,4,136,1,1,-184,-3.9098,-3.0866,-2.5453,-8.4382,-9.8238,...,-0.9082,-5.5007,-0.6975,-3.0212,-2.2171,-177.3466,41.4210,21.8185,-66.5214,1.0


In [0]:
df_one_hot_encoded = pd.get_dummies(df)
df = df_one_hot_encoded

In [41]:
df_1 = df.groupby(np.arange(len(df))//25).mean()
df_1.head(60)

,Unnamed: 0,Subject,Block,Trial,Time,O2,O1,OZ,PZ,P4,...,FC3,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value
0,12,136,1,1,-152,0.375268,0.493672,0.543364,-2.324324,-2.157836,...,1.955744,-0.438476,1.599680,-0.427660,-0.126032,20.553288,-37.434624,19.033260,-18.427808,1.0
1,37,136,1,1,-52,-0.375264,-0.493664,-0.543360,2.324324,2.157820,...,-1.955740,0.438480,-1.599684,0.427652,0.126040,-20.553280,37.434636,-19.033256,18.427816,1.0
2,62,136,1,1,48,-11.409724,-7.042056,-8.682444,0.712524,-4.275536,...,-3.261480,-3.496132,-8.217220,-8.431116,-7.408164,-214.309440,127.380020,-45.196156,-7.477256,1.0
3,87,136,1,1,148,-8.498108,-1.825220,-5.125564,0.957016,-4.746636,...,-3.878444,-7.556604,-10.603796,-12.219608,-7.979536,-250.093680,149.466544,-38.970288,-16.557496,1.0
4,112,136,1,1,248,-9.932424,-5.155828,-8.878360,-1.857008,-3.561316,...,2.990128,0.182492,-2.856056,-4.131232,-0.428140,-256.681804,158.501428,-17.493732,-20.887424,1.0
5,137,136,1,1,348,-4.947772,7.563476,-1.030220,-3.110852,-4.645244,...,0.483724,-2.481212,-9.350956,-8.291592,-5.550304,-276.813444,155.818424,-23.572948,-32.203196,1.0
6,162,136,1,1,448,-14.802232,3.980400,-7.970940,-13.048228,-14.982392,...,-9.581812,-10.248836,-11.655608,-8.590824,-6.880652,-297.404456,159.448696,-30.242556,-30.573288,1.0
7,187,136,1,1,548,-22.339756,0.067492,-13.867188,-16.948916,-19.311348,...,-2.839660,1.821988,0.031004,0.475096,1.621516,-276.474772,164.621344,-14.384364,-22.476884,1.0
8,212,136,1,1,648,-19.152156,3.819820,-10.638768,-12.393580,-16.413368,...,-3.983688,-2.358504,-4.049400,-4.192456,-2.662552,-279.754304,161.095024,-17.949276,-23.254840,1.0
9,237,136,1,1,748,-21.821908,0.478692,-14.395632,-15.202620,-16.966836,...,-2.367228,-1.623508,-1.360236,-2.604364,-0.096236,-281.883452,167.816476,-13.759828,-21.410392,1.0


In [6]:
import random
random_samp_list = []
index_list = []
for i in range(1,201):
  index_list.append(i)


train_samp_list = random.sample(index_list,k=100)
  
test_list = []
test_list = [x for x in index_list if x not in train_samp_list]


#df_train = df['Trial'] == i
print(train_samp_list)
print(test_list)

[117, 111, 177, 158, 13, 76, 123, 3, 58, 136, 173, 4, 121, 94, 24, 28, 31, 89, 164, 149, 30, 99, 141, 133, 51, 101, 106, 1, 27, 199, 165, 14, 200, 65, 95, 172, 23, 166, 80, 10, 72, 19, 46, 184, 178, 54, 159, 2, 34, 40, 20, 182, 128, 145, 160, 143, 161, 150, 174, 17, 189, 191, 43, 153, 63, 41, 84, 131, 115, 179, 91, 156, 45, 37, 98, 127, 42, 67, 103, 126, 196, 60, 190, 194, 7, 59, 116, 147, 11, 35, 168, 132, 144, 118, 78, 25, 151, 146, 49, 162]
[5, 6, 8, 9, 12, 15, 16, 18, 21, 22, 26, 29, 32, 33, 36, 38, 39, 44, 47, 48, 50, 52, 53, 55, 56, 57, 61, 62, 64, 66, 68, 69, 70, 71, 73, 74, 75, 77, 79, 81, 82, 83, 85, 86, 87, 88, 90, 92, 93, 96, 97, 100, 102, 104, 105, 107, 108, 109, 110, 112, 113, 114, 119, 120, 122, 124, 125, 129, 130, 134, 135, 137, 138, 139, 140, 142, 148, 152, 154, 155, 157, 163, 167, 169, 170, 171, 175, 176, 180, 181, 183, 185, 186, 187, 188, 192, 193, 195, 197, 198]


In [0]:
rfc = RandomForestClassifier(n_estimators=150, max_features='auto')
#rfc.fit(x_pd, y_pd);

In [40]:
#df_1 = df_1.drop(['Unnamed: 0','Subject'], axis=1)
df_2.head()

,Block,Trial,Time,PZ,P4,CP4,C4,T8,P3,CP3,CPZ,CZ,C3,T7,FC3,HEOL,Value
16,1.0,1.0,458.0,-12.24241,-15.57169,-16.93784,-20.26947,-22.44105,-6.60078,-6.28642,-14.75726,-14.94575,-8.80270,-5.71786,-8.47089,-29.93138,1.0
17,1.0,1.0,498.0,-10.46423,-10.53390,-10.48699,-14.13909,-16.39804,-3.47727,-5.09630,-13.51151,-14.36648,-8.46154,-5.74508,-7.91407,-31.29349,1.0
18,1.0,1.0,538.0,-13.97902,-17.53564,-13.44836,-11.88707,-22.37556,-3.48393,-2.39915,-12.31359,-8.71636,-3.00707,-1.10817,-1.04283,-16.05989,1.0
19,1.0,1.0,578.0,-24.22875,-26.90155,-22.80506,-22.32186,-25.73137,-10.60711,-6.36840,-20.54387,-17.32525,-4.71904,1.28439,-3.48529,-24.74505,1.0
20,1.0,1.0,618.0,-12.78559,-15.95735,-13.42807,-13.38168,-23.94478,-3.81356,-3.55739,-11.34967,-10.30545,-6.50206,-3.67540,-5.21318,-24.95480,1.0


In [39]:
from sklearn.model_selection import GroupShuffleSplit
n_splits = 20
cv = GroupShuffleSplit(n_splits=n_splits, test_size=0.2)
df_2 = df_1[(df_1['Value']==1) | (df_1['Value']==9)]
#df_2 = df_2.drop(['O1','O2','OZ','FP1','FP2','F3','FZ','F4','P8','TP8','P7','FC4','FT8','TP7','FCZ','F8','F7','FT7','VEOU','VEOL','HEOR','HEOR'], axis=1)
df_2 = df_2.drop(['VEOU','VEOL','HEOR','HEOR'], axis=1)
#df_2 = df_2[df_2['Time'] < 650]
#df_2 = df_2[df_2['Time'] > 450]
r_in = np.zeros(n_splits)
r_out = np.zeros(n_splits)
for i,(train, test) in enumerate(cv.split(df_2, groups=df_2['Trial'])):
  #train = df_2.index[train]
  #test = df_2.index[test]
  y = df_2['Value']
  X = df_2.drop('Value', axis=1).astype('float')
  rfc.fit(X.iloc[train].values, y.iloc[train])
  predicted = rfc.predict(X.iloc[train])
  actual = y.iloc[train]
  r_in[i] = np.corrcoef(predicted, actual)[0, 1]
  t_predicted = rfc.predict(X.iloc[test])
  t_actual = y.iloc[test]
  r_out[i] = np.corrcoef(t_predicted, t_actual)[0, 1]
  print(r_in[i], r_out[i])
print("Mean R_out = %.3g +/- %.3g" % (r_out.mean(), r_out.std()))


1.0 0.1632993161855451
0.9999999999999999 0.12259495208163808
1.0 -0.08257950118786417
1.0 0.2847607143234435
1.0 -0.06856232303828141
1.0 -0.00827152778309103
1.0 0.13981473517895088
1.0 0.1641219879724435
1.0 0.18955465781171954
1.0 0.03382308596212273
1.0 -0.14232464388839292
1.0 0.0020104777662893142
1.0 -0.057166195047502824
1.0 0.09070796093247058
1.0 0.2211629342323457
1.0 0.2984201070484202
1.0 0.1591309874160419
1.0 0.08164965809277261
1.0 0.3806858523579086
0.9999999999999999 0.21821789023599236
Mean R_out = 0.11 +/- 0.136


In [0]:
plt.scatter(t_predicted,predicted)

In [0]:
df_train = df.drop(df.index[train_samp_list])
df_test = df.drop(df.index[test_list])

    
df_x = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)
df_y = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)

In [0]:
y_pd = df_y['Value']
x_pd = df_x

In [0]:
y_pd_predicted = rfc.predict(x_pd)
#formula
#def report(y, y_predicted):
 # conf_mat = confusion_marix(y, y_predicted)
  #report = pd.DataFrame(index=['Value'])
  
print(y_pd_predicted)

[1. 1. 1. ... 3. 3. 3.]
